# A Semi-Automated Way to Run GHOST DR 


**Some errors / warning messages**  
They don't seem to hinder actual operation, but will check with Chris or Fletcher about these errors at some point

```
  PRIMITIVE: biasCorrect
   ----------------------
   ERROR - Inputs have different numbers of SCI extensions.
```


-----

```
   PRIMITIVE: processSlits
   -----------------------
      PRIMITIVE: getProcessedSlitFlat
      -------------------------------
         PRIMITIVE: getCalibration
         -------------------------
         Received calibration request for arcs_sr_1x1_brs300_20220629_2x2_slit_varAdded.fits
         LOCAL CALIBRATION SEARCH:
         WARNING - START CALIBRATION SERVICE REPORT
         WARNING - 
         WARNING - 	Could not find a proper calibration in the local database
         WARNING - 	No processed_slitflat calibration file found for arcs_sr_1x1_brs300_20220629_2x2_slit_varAdded.fits
         WARNING - 
         WARNING - END CALIBRATION SERVICE REPORT
         WARNING - 
         .
      WARNING - Calibration not found for arcs_sr_1x1_brs300_20220629_2x2_slit_varAdded.fits
      .
   WARNING - Unable to find slitflat calibration for arcs_sr_1x1_brs300_20220629_2x2_slit_varAdded.fits; skipping
 ```
 
 -----
 Got overscan correct files for blue flats but not red flats ? What did I do ?
 
 ----
 
      PRIMITIVE: getCalibration
         -------------------------
         Received calibration request for aqu2_sr_2x4_br1800s300_20220629_2x4_red003_arraysTiled.fits
         LOCAL CALIBRATION SEARCH:
         WARNING - START CALIBRATION SERVICE REPORT
         WARNING - 
         WARNING - 	Could not find a proper calibration in the local database
         WARNING - 	No processed_flat calibration file found for aqu2_sr_2x4_br1800s300_20220629_2x4_red003_arraysTiled.fits
         WARNING - 
         WARNING - END CALIBRATION SERVICE REPORT
         WARNING - 
         .
      WARNING - Calibration not found for aqu2_sr_2x4_br1800s300_20220629_2x4_red003_arraysTiled.fits
      .
   WARNING - No flat identified/provided for aqu2_sr_2x4_br1800s300_20220629_2x4_red003_arraysTiled.fits - skipping

In [1]:
import numpy as np
import os
import sys
import subprocess
import stat
import shutil
import glob

In [2]:
def caldb_list():
    file  = open('./caldbList.sh','w')
    file.write('#!/bin/sh \nsource activate dragons\ncaldb list')
    file.close()
    os.chmod('caldbList.sh', stat.S_IRWXU )

    _ = subprocess.call('./caldbList.sh')

# Step 0) Set up

This notebook does not have to be in the same folder as your data, but it does assume some things about your data structure.   
- For each object you want to reduce, create a folder for it (ex. I will create Aq2_star1)
- In that folder (ex. Aq2_star1), create another folder with all the packed data for this reduction called "packed". (ex.  Aq2_star1 > packed )  
Thats it. The notebook will handle the rest of the data organization. 


### Change the cal manager

If you need to, change the cal manager.

In a terminal 

```bash
cd ~/.geminidr
nano rsys.cfg
```


### Change IFU Settings?

- If you have two objects in std-res you will need to reduce your object spectrum twice, once for IFU 0 and once for IFU 1.

Need to open the following code and hack it:
'~GHOSTDR/ghostdr/ghost/primitives_ghost_spect.py'


 
Find “objs_to_use” and change to below for IFU0 and no sky: 
```
          	  if 'ARC' in ad.tags:
              	 objs_to_use = [[0], ]
               	 use_sky = [False, ]
```
also change
```
            else:
                objs_to_use = [[0],  ]
                use_sky = [ False, ]
```

Find “objs_to_use” and change to below for IFU1 and no sky: 
```
          	  if 'ARC' in ad.tags:
              	 objs_to_use = [[0,1], ]
               	 use_sky = [False, ]
```
also change
```
For 2-object mode (std) no sky  
            else:
                objs_to_use = [  [0, 1],  ]
                use_sky = [   False, ]
```

# Step 1)  Move to Directory With Data

In [3]:
# -------------- Move into the directory with your data
# Path to where your data is located 
Path = '~/Ghost/GHOST_ObsData/Aq2'  # CHANGE ME
os.chdir(Path)

**Enter required information about your data**

In [4]:
# The name of the slit view image you want to use
goodSlitView = 'HD122196_HIP068460_sr_1x1_br300s3_2x2_slit.fits' # CHANGE ME

# The name of the slit arc you want to us
SlitArc = 'arcs_sr_1x1_brs300_20220629_2x2_slit.fits' # CHANGE ME

# The name of the binning mode of your science data 
binningMode = '2x4' # CHANGE ME

# The resolution of your data
res = 'STD'# CHANGE ME

# The name of your science frame (the first set of characters is fine)
scienceObj = 'aqu2_sr_2x4_br1800s300_20220629' # Change Me

**If you need to fix your fits files / header info**

In [ ]:
# Path to where your fits scripts are located
fixFitsScriptPath = '~/GHOST/GHOSTDR/' 

Theoretically, this is all you need to change. Now just start running the cells

# Step 2) Initialize caldb 
Can skip this step if you've done this already


Overview / Example:
```bash
caldb init -v -w
```

In [5]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Create a shell file and run the first command - conda init -v -w
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\ncaldb init -v -w')
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')

Initializing /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/cal_manager.db...


# Step 2) Unpack the Data

Overview / Example:
```bash
reduce --drpkg ghostdr *.fits
```

In [6]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# If no raw folder has been created yet:
# -------------- Create a folder called raw
if os.path.isdir('raw') == False:
    os.mkdir('raw')

    # -------------- Create a shell file and run the command to unpack the data
    file  = open('./script.sh','w')
    file.write('#!/bin/sh \nsource activate dragons\nreduce --drpkg ghostdr packed/*.fits')
    file.close()
    os.chmod('script.sh', stat.S_IRWXU )

    _ = subprocess.call('./script.sh')

    # -------------- Move the unpacked data into raw
    for file in glob.glob('*.fits'):
        shutil.move(file, 'raw/')


			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
packed/HD122196_HIP068460_sr_1x1_br300s3.fits ... packed/flat_sr_1x1_br6_s02_20220629.fits, 6 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedBundle
   PRIMITIVE: splitBundle
   ----------------------
   Unbundling HD122196_HIP068460_sr_1x1_br300s3.fits:
      Writing HD122196_HIP068460_sr_1x1_br300s3_2x2_slit.fits
      Writing HD122196_HIP068460_sr_1x1_br300s3_1x1_blue001.fits
      Writing HD122196_HIP068460_sr_1x1_br300s3_1x1_red001.fits
   Unbundling aqu2_sr_2x4_br1800s300_20220629.fits:
      Writing aqu2_sr_2x4_br1800s300_20220629_2x2_slit.fits
      Writing aqu2_sr_2x4_br1800s300_20220629_2x4_blue001.fits
      Writing aqu2_sr_2x4_br1800s300_20220629_2x4_blue002.fits
      Writing aqu2_sr_2x4_br1800s300_20220629_2x4_blue003.fits
      Writing aqu2_sr_2x4_br1800s300_20220629_2x4_red001.fits
      Writing aqu2_sr_2x4_br1800s300_20220629_2x4_red002.fits
      Writing aqu2_sr_2x4_br1800s300_20220629_2x4_red003.fits
   Unbundling arcs_sr_1x1_brs300_20220629.fits:
      Writing arcs_sr_1x1_brs300_20220629_2x2_slit.fits
      Writing arcs_sr_1x1_brs300_20220629_1x1_blue001.fits
      Writing arcs_sr_1x1

### Fix Data If Needed

- Skip this cell if your files dont need to be fixed 

In [7]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Temporarily Move into Raw
os.chdir(Path+'/raw')

# -------------- Paths to your scripts
scripts = ['correct_fits2.py', 'correct_slitbias.py', 'correct_gain_read_blue.py', 'correct_gain_read_red.py' ]

# -------------- Iterate through scripts and fix things
for script in scripts:
    print('\nFiles Modified By '+script+'\n')
    os.chmod(fixFitsScriptPath+script, stat.S_IRWXU )
    subprocess.call('python '+fixFitsScriptPath+script, shell=True)

# -------------- Move back out 
os.chdir(Path)

Files Modified By correct_fits2.py

HD122196_HIP068460_sr_1x1_br300s3_1x1_red001.fits
flat_sr_1x1_br6_s02_20220629_1x1_red001.fits
bias_2x4_20220629_2x4_red004.fits
HD122196_HIP068460_sr_1x1_br300s3_1x1_blue001.fits
bias_1x1_20220629_1x1_blue001.fits
aqu2_sr_2x4_br1800s300_20220629_2x4_red003.fits
aqu2_sr_2x4_br1800s300_20220629_2x4_red002.fits
bias_2x4_20220629_2x4_red005.fits
bias_1x1_20220629_2x2_slit.fits
bias_2x4_20220629_2x4_blue001.fits
arcs_sr_1x1_brs300_20220629_1x1_red003.fits
arcs_sr_1x1_brs300_20220629_1x1_blue002.fits
bias_2x4_20220629_2x4_red002.fits
aqu2_sr_2x4_br1800s300_20220629_2x2_slit.fits
flat_sr_1x1_br6_s02_20220629_2x2_slit.fits
flat_sr_1x1_br6_s02_20220629_1x1_blue001.fits
aqu2_sr_2x4_br1800s300_20220629_2x4_blue001.fits
bias_1x1_20220629_1x1_red001.fits
bias_2x4_20220629_2x4_red003.fits
arcs_sr_1x1_brs300_20220629_1x1_blue003.fits
arcs_sr_1x1_brs300_20220629_1x1_red002.fits
bias_1x1_20220629_1x1_blue005.fits
bias_1x1_20220629_1x1_red002.fits
arcs_sr_1x1_brs300_

# Step 3) Generate Slit View Frames

### 3.1 Slit View Bias

Overview / Example:
```bash
typewalk --adpkg ghost_instruments --tags GHOST BIAS SLITV --dir raw -o slit.bias.list
reduce --drpkg ghostdr @slit.bias.list
caldb add calibrations/processed_bias/ ....
```

In [8]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST BIAS SLITV --dir raw -o slit.bias.list'
l2 = 'reduce --drpkg ghostdr @slit.bias.list'

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')

# ------------------------------------------ 
# This produced a processed slit bias. 

# -------------- Note the name
processedSlitBias =  glob.glob('*.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed slit bias into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedSlitBias)):
    os.remove(os.path.join('temp/', processedSlitBias))
shutil.move(processedSlitBias, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_bias/'+processedSlitBias
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     bias_1x1_20220629_2x2_slit.fits ... (2x2) (BIAS) (CAL) (GEMINI) (GHOST) (IMAGE) (RAW) (SLIT) (SLITV) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x2_slit.fits ... (2x2) (BIAS) (CAL) (GEMINI) (GHOST) (IMAGE) (RAW) (SLIT) (SLITV) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_1x1_20220629_2x2_slit.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_2x4_20220629_2x2_slit.fits, 2 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedSlitBias
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No static BPMs defined
   .
   PRIMITIVE: addVAR
   -----------------
   Adding the read noise component of the variance
   .
   PRIMITIVE: stackFrames
   ----------------------
      

**Check caldb contents**

In [9]:
caldb_list()

bias_1x1_20220629_2x2_slit_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias


### 3.2 Slit View Flats

Overview / Example:
```bash
typewalk --adpkg ghost_instruments --tags GHOST SLITV FLAT STD --dir raw -o slit.flat.std.list
reduce --drpkg ghostdr -p biasCorrect:bias=.... @slit.flat.std.list
caldb add calibrations/processed_slitflat/ ....
```

Don't worry, the code with automatically use the bias created above

In [10]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST SLITV FLAT {} --dir raw -o slit.flat.{}.list'.format(res,res)
l2 = 'reduce --drpkg ghostdr -p biasCorrect:bias=calibrations/processed_bias/{} @slit.flat.{}.list'.format(processedSlitBias,res)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')

# ------------------------------------------ 
# This produced a processed slit flat. 

# -------------- Note the name
processedSlitFlat =  glob.glob('*slitflat.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed slit bias into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedSlitFlat)):
    os.remove(os.path.join('temp/', processedSlitFlat))
shutil.move(processedSlitFlat, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_slitflat/'+processedSlitFlat
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     flat_sr_1x1_br6_s02_20220629_2x2_slit.fits
     ................................... (2x2) (CAL) (FLAT) (GEMINI) (GHOST) (IMAGE) (RAW) (SLIT) (SLITFLAT) (SLITV) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/flat_sr_1x1_br6_s02_20220629_2x2_slit.fits
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedSlitFlat
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   .
   PRIMITIVE: addVAR
   -----------------
   Adding the read noise component of the variance
   .
   PRIMITIVE: biasCorrect
   ----------------------
   ERROR - Inputs have different numbers of SCI extensions.
   .
   PRIMITIVE: ADUToElectrons
   -------

In [12]:
caldb_list()

bias_1x1_20220629_2x2_slit_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
flat_sr_1x1_br6_s02_20220629_2x2_slit_slitflat.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_slitflat


### 3.3 Slit View Image

Overview / Example:
```bash
reduce --drpkg ghostdr -p biasCorrect:bias=... raw/...
caldb add calibrations/processed_slit/ ....
```

Don't worry, the code with automatically use the bias created above. You've just got to enter the name of the slit view image you want to use

**Note**
I added another primative which points to the slit flat file because it wasn't finding the correct slitflat 

In [14]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Create the Commands
l2 = 'reduce --drpkg ghostdr -p biasCorrect:bias=calibrations/processed_bias/{} processSlits:slitflat=calibrations/processed_slitflat/{} raw/{}'.format(processedSlitBias,processedSlitFlat, goodSlitView)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')

# ------------------------------------------ 
# This produced a processed slit view. 

# -------------- Note the name
processedSlit =  glob.glob('*slit.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed slit bias into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedSlit)):
    os.remove(os.path.join('temp/', processedSlit))
shutil.move(processedSlit, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_slit/'+processedSlit
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
raw/HD122196_HIP068460_sr_1x1_br300s3_2x2_slit.fits
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedSlit
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   .
   PRIMITIVE: addVAR
   -----------------
   Adding the read noise component of the variance
   .
   PRIMITIVE: biasCorrect
   ----------------------
   ERROR - Inputs have different numbers of SCI extensions.
   .
   PRIMITIVE: ADUToElectrons
   -----------

### 3.3 Slit View For the Arc

Overview / Example:
```bash
reduce --drpkg ghostdr -p biasCorrect:bias=.... raw/....
caldb add calibrations/processed_slit/ ....
```

Don't worry, the code with automatically use the bias created above. You've just got to enter the name of the slit arc you want to use

**Note**
I added another primative which (I think) points to the slit flat file because it wasn't finding the correct slitflat 

In [15]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Create the Commands
l2 = 'reduce --drpkg ghostdr -p biasCorrect:bias=calibrations/processed_bias/{} processSlits:slitflat=calibrations/processed_slitflat/{} raw/{}'.format(processedSlitBias,processedSlitFlat, SlitArc)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')

# ------------------------------------------ 
# This produced a processed slit Arc. 

# -------------- Note the name
processedSlitArc =  glob.glob('arcs*slit.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed slit bias into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedSlitArc)):
    os.remove(os.path.join('temp/', processedSlitArc))
shutil.move(processedSlitArc, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_slit/'+processedSlitArc
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
raw/arcs_sr_1x1_brs300_20220629_2x2_slit.fits
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedSlit
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   .
   PRIMITIVE: addVAR
   -----------------
   Adding the read noise component of the variance
   .
   PRIMITIVE: biasCorrect
   ----------------------
   ERROR - Inputs have different numbers of SCI extensions.
   .
   PRIMITIVE: ADUToElectrons
   -----------

In [16]:
caldb_list()

HD122196_HIP068460_sr_1x1_br300s3_2x2_slit_slit.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_slit
arcs_sr_1x1_brs300_20220629_2x2_slit_slit.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_slit
bias_1x1_20220629_2x2_slit_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
flat_sr_1x1_br6_s02_20220629_2x2_slit_slitflat.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_slitflat


Phew. Got this far. 

# Step 4) Generate Bias Calibration Frames

Overview / Example:
```bash
typewalk --adpkg ghost_instruments --tags GHOST BIAS RED 1x1 --dir raw -o bias.1x1.red.list
reduce --drpkg ghostdr @bias.1x1.red.list
caldb add calibrations/processed_bias/...
```

### 4.1 1x1 Biases

In [17]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST BIAS RED 1x1 --dir raw -o bias.1x1.red.list'
l2 = 'reduce --drpkg ghostdr -p @bias.1x1.red.list'

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------ 
# This produced a processed 1x1 red bias frame. 

# -------------- Note the name
processedBias_1x1Red =  glob.glob('bias*red*bias.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed bias into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedBias_1x1Red)):
    os.remove(os.path.join('temp/', processedBias_1x1Red))
shutil.move(processedBias_1x1Red, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_bias/'+processedBias_1x1Red
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------------------------------------------------ 
# Lets do it again for blue

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST BIAS BLUE 1x1 --dir raw -o bias.1x1.blue.list'
l2 = 'reduce --drpkg ghostdr -p @bias.1x1.blue.list'

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------ 
# This produced a processed 1x1 red bias frame. 

# -------------- Note the name
processedBias_1x1Blue =  glob.glob('bias*blue*bias.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed bias into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedBias_1x1Blue)):
    os.remove(os.path.join('temp/', processedBias_1x1Blue))
shutil.move(processedBias_1x1Blue, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_bias/'+processedBias_1x1Blue
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# -------------- Move the forStack bias frames to temp
processedBias_forStack =  glob.glob('bias*forStack.fits')

for item in processedBias_forStack:
    if os.path.isfile(os.path.join('temp/', item)):
        os.remove(os.path.join('temp/', item))
    shutil.move(item, 'temp/')

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     bias_1x1_20220629_1x1_red001.fits . (1x1) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     bias_1x1_20220629_1x1_red002.fits . (1x1) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     bias_1x1_20220629_1x1_red003.fits . (1x1) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     bias_1x1_20220629_1x1_red004.fits . (1x1) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     bias_1x1_20220629_1x1_red005.fits . (1x1) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_1x1_20220629_1x1_red001.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_1x1_20220629_1x1_red005.fits, 5 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedBias
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     bias_1x1_20220629_1x1_blue001.fits
     ................................... (1x1) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     bias_1x1_20220629_1x1_blue002.fits
     ................................... (1x1) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     bias_1x1_20220629_1x1_blue003.fits
     ................................... (1x1) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     bias_1x1_20220629_1x1_blue004.fits
     ................................... (1x1) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     bias_1x1_20220629_1x1_blue005.fits
     ................................... (1x1) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_1x1_20220629_1x1_blue001.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_1x1_20220629_1x1_blue005.fits, 5 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedBias
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No

### 4.2 nxm Biases

Now to do the biases for the science data.  You need to enter the data binning mode (shouldd have been done above)

Overview / Example:
```bash
typewalk --adpkg ghost_instruments --tags GHOST BIAS RED nxm --dir raw -o bias.nxm.red.list
reduce --drpkg ghostdr @bias.nxm.red.list
caldb add calibrations/processed_bias/...
```

In [18]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST BIAS RED {} --dir raw -o bias.{}.red.list'.format(binningMode, binningMode)
l2 = 'reduce --drpkg ghostdr -p @bias.{}.red.list'.format(binningMode)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------ 
# This produced a processed 1x1 red bias frame. 

# -------------- Note the name
processedBias_nxmRed =  glob.glob('bias*red*bias.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed bias into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedBias_nxmRed)):
    os.remove(os.path.join('temp/', processedBias_nxmRed))
shutil.move(processedBias_nxmRed, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_bias/'+processedBias_nxmRed
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------------------------------------------------ 
# Lets do it again for blue

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST BIAS BLUE {} --dir raw -o bias.{}.blue.list'.format(binningMode, binningMode)
l2 = 'reduce --drpkg ghostdr -p @bias.{}.blue.list'.format(binningMode)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------ 
# This produced a processed 1x1 red bias frame. 

# -------------- Note the name
processedBias_nxmBlue =  glob.glob('bias*blue*bias.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed bias into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedBias_nxmBlue)):
    os.remove(os.path.join('temp/', processedBias_nxmBlue))
shutil.move(processedBias_nxmBlue, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_bias/'+processedBias_nxmBlue
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# -------------- Move the forStack bias frames to temp
processedBias_forStack =  glob.glob('bias*forStack.fits')

for item in processedBias_forStack:
    if os.path.isfile(os.path.join('temp/', item)):
        os.remove(os.path.join('temp/', item))
    shutil.move(item, 'temp/')

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     bias_2x4_20220629_2x4_red001.fits . (2x4) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x4_red002.fits . (2x4) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x4_red003.fits . (2x4) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x4_red004.fits . (2x4) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x4_red005.fits . (2x4) (BIAS) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_2x4_20220629_2x4_red001.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_2x4_20220629_2x4_red005.fits, 5 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedBias
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     bias_2x4_20220629_2x4_blue001.fits
     ................................... (2x4) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x4_blue002.fits
     ................................... (2x4) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x4_blue003.fits
     ................................... (2x4) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x4_blue004.fits
     ................................... (2x4) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     bias_2x4_20220629_2x4_blue005.fits
     ................................... (2x4) (BIAS) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_2x4_20220629_2x4_blue001.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/bias_2x4_20220629_2x4_blue005.fits, 5 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedBias
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No

Got the biases done. Killing it.

In [19]:
caldb_list()

HD122196_HIP068460_sr_1x1_br300s3_2x2_slit_slit.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_slit
arcs_sr_1x1_brs300_20220629_2x2_slit_slit.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_slit
bias_1x1_20220629_1x1_blue001_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
bias_1x1_20220629_1x1_red001_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
bias_1x1_20220629_2x2_slit_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
bias_2x4_20220629_2x4_blue001_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
bias_2x4_20220629_2x4_red001_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
flat_sr_1x1_br6_s02_20220629_2x2_slit_slitflat.fits /Users/je

# Step 5) Generate the Flat Calibration Frames

In [21]:
# ---------------------------- There is (theoretically) nothing to change in this cell

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST RED FLAT {} --dir raw -o bias.1x1.red.list'.format(res)
l2 = 'reduce --drpkg ghostdr -p biasCorrect:bias=calibrations/processed_bias/{} @bias.1x1.red.list'.format(processedBias_1x1Red)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------ 
# This produced a processed 1x1 red flat frame. 

# -------------- Note the name
processedFlat_Red =  glob.glob('flat*red*flat.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed flat into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedFlat_Red)):
    os.remove(os.path.join('temp/', processedFlat_Red))
shutil.move(processedFlat_Red, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_flat/'+processedFlat_Red
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------------------------------------------------ 
# Lets do it again for blue

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST BLUE FLAT {} --dir raw -o bias.1x1.blue.list'.format(res)
l2 = 'reduce --drpkg ghostdr -p biasCorrect:bias=calibrations/processed_bias/{} @bias.1x1.blue.list'.format(processedBias_1x1Blue)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------ 
# This produced a processed 1x1 blue flat frame. 

# -------------- Note the name
processedFlat_Blue =  glob.glob('flat*blue*flat.fits')[0]

# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed flat into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedFlat_Blue)):
    os.remove(os.path.join('temp/', processedFlat_Blue))
shutil.move(processedFlat_Blue, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_flat/'+processedFlat_Blue
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# -------------- Move the overscan Correct frames to temp
processedFlat_overscan =  glob.glob('flat*overscanCorrect.fits')

for item in processedFlat_overscan:
    if os.path.isfile(os.path.join('temp/', item)):
        os.remove(os.path.join('temp/', item))
    shutil.move(item, 'temp/')

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     flat_sr_1x1_br6_s02_20220629_1x1_red001.fits
     ................................... (1x1) (CAL) (FLAT) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     flat_sr_1x1_br6_s02_20220629_1x1_red002.fits
     ................................... (1x1) (CAL) (FLAT) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     flat_sr_1x1_br6_s02_20220629_1x1_red003.fits
     ................................... (1x1) (CAL) (FLAT) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     flat_sr_1x1_br6_s02_20220629_1x1_red004.fits
     ................................... (1x1) (CAL) (FLAT) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     flat_sr_1x1_br6_s02_20220629_1x1_red005.fits
     ................................... (1x1) (CAL) (FLAT) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/flat_sr_1x1_br6_s02_20220629_1x1_red001.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/flat_sr_1x1_br6_s02_20220629_1x1_red005.fits, 5 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedFlat
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No

Fitting (this can sometimes take a while...)
[ 3742.01503958 15111.17760134  3779.4615575  ...  5752.98725474
  5631.79246818  5600.05405874] [-6987.79878357  4129.95951241 -7354.08483157 ...  -606.92271912
  -620.05811388  -642.438972  ]
Creating order: 0
Creating order: 1
Creating order: 2
Creating order: 3
Creating order: 4
Creating order: 5
Creating order: 6
Creating order: 7
Creating order: 8
Creating order: 9
Creating order: 10
Creating order: 11
Creating order: 12
Creating order: 13
Creating order: 14
Creating order: 15
Creating order: 16
Creating order: 17
Creating order: 18
Creating order: 19
Creating order: 20
Creating order: 21
Creating order: 22
Creating order: 23
Creating order: 24
Creating order: 25
Creating order: 26
Creating order: 27
Creating order: 28
Creating order: 29
Creating order: 30
Creating order: 31
Creating order: 32


...


			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/flat_sr_1x1_br6_s02_20220629_1x1_blue001.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/flat_sr_1x1_br6_s02_20220629_1x1_blue005.fits, 5 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedFlat
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No

Fitting (this can sometimes take a while...)
[-1101.05246134 -1122.17292187 -5646.49034006 ...   -44.71684954
   -90.10142384  -100.72608531] [ 1733.8909474   1793.56479189 -2685.87796907 ...   115.53700134
    72.24711277    72.17042274]
Creating order: 0
Creating order: 1
Creating order: 2
Creating order: 3
Creating order: 4
Creating order: 5
Creating order: 6
Creating order: 7
Creating order: 8
Creating order: 9
Creating order: 10
Creating order: 11
Creating order: 12
Creating order: 13
Creating order: 14
Creating order: 15
Creating order: 16
Creating order: 17
Creating order: 18
Creating order: 19
Creating order: 20
Creating order: 21
Creating order: 22
Creating order: 23
Creating order: 24
Creating order: 25
Creating order: 26
Creating order: 27
Creating order: 28
Creating order: 29
Creating order: 30
Creating order: 31
Creating order: 32
Creating order: 33
Creating order: 34


   .
   PRIMITIVE: storeProcessedFlat
   -----------------------------
      PRIMITIVE: storeCalibration
      ---------------------------
      Calibration stored as calibrations/processed_flat/flat_sr_1x1_br6_s02_20220629_1x1_blue001_flat.fits
      .
   .
	Wrote flat_sr_1x1_br6_s02_20220629_1x1_blue001_flat.fits in output directory

reduce completed successfully.


In [22]:
caldb_list()

HD122196_HIP068460_sr_1x1_br300s3_2x2_slit_slit.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_slit
arcs_sr_1x1_brs300_20220629_2x2_slit_slit.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_slit
bias_1x1_20220629_1x1_blue001_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
bias_1x1_20220629_1x1_red001_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
bias_1x1_20220629_2x2_slit_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
bias_2x4_20220629_2x4_blue001_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
bias_2x4_20220629_2x4_red001_bias.fits /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/calibrations/processed_bias
flat_sr_1x1_br6_s02_20220629_1x1_blue001_flat.fits /Users/jen

Almost there...

# Step 6) Generate the Arc Calibration Frames

This is only for a single arc (not before and after arcs, youre on your own for that)

### 6.1) Do the red arc

In [59]:
# ---------------------------- There is (theoretically) nothing to change in this cell

biasCorrect = 'calibrations/processed_bias/'+processedBias_1x1Red
extractProfile_slit = 'calibrations/processed_slit/'+processedSlitArc
extractProfile_flat = 'calibrations/processed_flat/'+processedFlat_Red
extractProfile_slitflat = 'calibrations/processed_slitflat/'+processedSlitFlat
fitWavelength_flat = 'calibrations/processed_flat/'+processedFlat_Red

primitives = 'biasCorrect:bias={} extractProfile:slit={} extractProfile:flat={} extractProfile:slitflat={} fitWavelength:flat={} fitWavelength:return_residuals=True'.format(biasCorrect, extractProfile_slit, extractProfile_flat, extractProfile_slitflat, fitWavelength_flat) 

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST ARC SPECT RED {} --dir raw -o arc.red.{}.spect.list'.format(res, res)
l2 = 'reduce --drpkg ghostdr -p {} @arc.red.{}.spect.list'.format(primitives, res)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------ 
# This produced a processed red arc frame. 

# -------------- Note the name
processedArc_Red =  glob.glob('arc*red*arc.fits')[0]


# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed arc into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedArc_Red)):
    os.remove(os.path.join('temp/', processedArc_Red))
shutil.move(processedArc_Red, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_arc/'+processedArc_Red
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# -------------- Move the overscanCorrect / Tiled / Profile frames to temp
processedArc_2temp =  glob.glob('arc*overscanCorrect.fits') + glob.glob('arc*arraysTiled.fits') + glob.glob('arc*extractedProfile.fits')

for item in processedArc_2temp:
    if os.path.isfile(os.path.join('temp/', item)):
        os.remove(os.path.join('temp/', item))
    shutil.move(item, 'temp/')

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     arcs_sr_1x1_brs300_20220629_1x1_red001.fits
     ................................... (1x1) (ARC) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     arcs_sr_1x1_brs300_20220629_1x1_red002.fits
     ................................... (1x1) (ARC) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
     arcs_sr_1x1_brs300_20220629_1x1_red003.fits
     ................................... (1x1) (ARC) (CAL) (GEMINI) (GHOST) (RAW) (RED) (SPECT) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/arcs_sr_1x1_brs300_20220629_1x1_red001.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/arcs_sr_1x1_brs300_20220629_1x1_red003.fits, 3 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedArc
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No static BPMs defined
   WARNING - No static BPMs defined
   .
   PRIMITIVE: addVAR
   -----------------
   Adding the read no

Creating order: 0
Creating order: 1
Creating order: 2
Creating order: 3
Creating order: 4
Creating order: 5
Creating order: 6
Creating order: 7
Creating order: 8
Creating order: 9
Creating order: 10
Creating order: 11
Creating order: 12
Creating order: 13
Creating order: 14
Creating order: 15
Creating order: 16
Creating order: 17
Creating order: 18
Creating order: 19
Creating order: 20
Creating order: 21
Creating order: 22
Creating order: 23
Creating order: 24
Creating order: 25
Creating order: 26
Creating order: 27
Creating order: 28
Creating order: 29
Creating order: 30
Creating order: 31
Creating order: 32
OBJECTS:[0, 1]
SKY:False
Extracting order: 0
Extracting order: 1
Extracting order: 2
Extracting order: 3
Extracting order: 4
Extracting order: 5
Extracting order: 6
Extracting order: 7
Extracting order: 8
Extracting order: 9
Extracting order: 10
Extracting order: 11
Extracting order: 12
Extracting order: 13
Extracting order: 14
Extracting order: 15
Extracting order: 16
Extracting 

   .
   PRIMITIVE: fitWavelength
   ------------------------


order  0
order  1
order  2
Rejecting due to low SNR!
order  3
Rejecting due to low SNR!
Rejecting due to low SNR!
order  4
Rejecting due to low SNR!
Rejecting due to low SNR!
order  5
order  6
Rejecting due to low SNR!
order  7
order  8
order  9
order  10
Rejecting due to low SNR!
order  11
order  12
order  13
order  14
order  15
order  16
order  17
order  18
order  19
order  20
order  21
Rejecting due to low SNR!
order  22
Rejecting due to low SNR!
order  23
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
order  24
Rejecting due to low SNR!
order  25
Rejecting due to low SNR!
Rejecting due to low SNR!
order  26
Rejecting due to low SNR!
order  27
Rejecting due to low SNR!
order  28
Rejecting due to low SNR!
order  29
order  30
Rejecting due to low SNR!
Rejecting due to low SNR!
order  31
Rejecting due to low SNR!
Rejecting due to low SNR!
order  32
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
Reje

   .
   PRIMITIVE: storeProcessedArc
   ----------------------------
      PRIMITIVE: storeCalibration
      ---------------------------
      Calibration stored as calibrations/processed_arc/arcs_sr_1x1_brs300_20220629_1x1_red001_arc.fits
      .
   .
	Wrote arcs_sr_1x1_brs300_20220629_1x1_red001_arc.fits in output directory

reduce completed successfully.


**Note the residual in the output from above**
- Ideally residual <= 0.005, but <= 0.007 is also acceptable


Check the file: `wave_resid_std_red.txt`  
So once you have found a line that is bad, what you can do is go to your   
`GHOSTDR > ghostdr > ghost > lookups > Polyfit > ghost_thar_linelist_20220718.txt`  
find the bad line(s) and comment them out.  

Now when you rerun the cell above you should get a better residual.  

### 6.2) Do the blue arc

In [63]:
# ---------------------------- There is (theoretically) nothing to change in this cell

biasCorrect = 'calibrations/processed_bias/'+processedBias_1x1Blue
extractProfile_slit = 'calibrations/processed_slit/'+processedSlitArc
extractProfile_flat = 'calibrations/processed_flat/'+processedFlat_Blue
extractProfile_slitflat = 'calibrations/processed_slitflat/'+processedSlitFlat
fitWavelength_flat = 'calibrations/processed_flat/'+processedFlat_Blue

primitives = 'biasCorrect:bias={} extractProfile:slit={} extractProfile:flat={} extractProfile:slitflat={} fitWavelength:flat={} fitWavelength:return_residuals=True'.format(biasCorrect, extractProfile_slit, extractProfile_flat, extractProfile_slitflat, fitWavelength_flat) 

# -------------- Create the Commands
l1 = 'typewalk --adpkg ghost_instruments --tags GHOST ARC SPECT BLUE {} --dir raw -o arc.blue.{}.spect.list'.format(res, res)
l2 = 'reduce --drpkg ghostdr -p {} @arc.blue.{}.spect.list'.format(primitives, res)

# -------------- Add above commands to a script and run it
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l1+'\n'+l2)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# ------------------------------------------ 
# This produced a processed red arc frame. 

# -------------- Note the name
processedArc_Blue =  glob.glob('arc*blue*arc.fits')[0]


# -------------- Create a folder called temp if its not already been created
if os.path.isdir('temp') == False:
    os.mkdir('temp')
    
# -------------- Move the processed arc into temp.  Don't worry another copy is already in calibrations
if os.path.isfile(os.path.join('temp/', processedArc_Blue)):
    os.remove(os.path.join('temp/', processedArc_Blue))
shutil.move(processedArc_Blue, 'temp/')

# -------------- Add commands to a script and run it
l = 'caldb add calibrations/processed_arc/'+processedArc_Blue
file  = open('./script.sh','w')
file.write('#!/bin/sh \nsource activate dragons\n'+l)
file.close()
os.chmod('script.sh', stat.S_IRWXU )

_ = subprocess.call('./script.sh')


# -------------- Move the overscanCorrect / Tiled / Profile frames to temp
processedArc_2temp =  glob.glob('arc*overscanCorrect.fits') + glob.glob('arc*arraysTiled.fits') + glob.glob('arc*extractedProfile.fits')

for item in processedArc_2temp:
    if os.path.isfile(os.path.join('temp/', item)):
        os.remove(os.path.join('temp/', item))
    shutil.move(item, 'temp/')

ghost_instruments

directory:  /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw
     arcs_sr_1x1_brs300_20220629_1x1_blue001.fits
     ................................... (1x1) (ARC) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     arcs_sr_1x1_brs300_20220629_1x1_blue002.fits
     ................................... (1x1) (ARC) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
     arcs_sr_1x1_brs300_20220629_1x1_blue003.fits
     ................................... (1x1) (ARC) (BLUE) (CAL) (GEMINI) (GHOST) (RAW) (SPECT) (STD) (UNPREPARED) 
Done DataSpider.typewalk(..)



			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
/Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/arcs_sr_1x1_brs300_20220629_1x1_blue001.fits ... /Users/jennifer/Desktop/School/Year_6/Phys499/GHOST_ObsData/Aq2/raw/arcs_sr_1x1_brs300_20220629_1x1_blue003.fits, 3 files submitted.
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: makeProcessedArc
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   WARNING - No static BPMs defined
   WARNING - No static BPMs defined
   .
   PRIMITIVE: addVAR
   -----------------
   Adding the read no

Creating order: 0
Creating order: 1
Creating order: 2
Creating order: 3
Creating order: 4
Creating order: 5
Creating order: 6
Creating order: 7
Creating order: 8
Creating order: 9
Creating order: 10
Creating order: 11
Creating order: 12
Creating order: 13
Creating order: 14
Creating order: 15
Creating order: 16
Creating order: 17
Creating order: 18
Creating order: 19
Creating order: 20
Creating order: 21
Creating order: 22
Creating order: 23
Creating order: 24
Creating order: 25
Creating order: 26
Creating order: 27
Creating order: 28
Creating order: 29
Creating order: 30
Creating order: 31
Creating order: 32
Creating order: 33
Creating order: 34
OBJECTS:[0, 1]
SKY:False
Extracting order: 0
Extracting order: 1
Extracting order: 2
Extracting order: 3
Extracting order: 4
Extracting order: 5
Extracting order: 6
Extracting order: 7
Extracting order: 8
Extracting order: 9
Extracting order: 10
Extracting order: 11
Extracting order: 12
Extracting order: 13
Extracting order: 14
Extracting orde

   .
   PRIMITIVE: fitWavelength
   ------------------------


order  0
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
order  1
Rejecting due to low SNR!
order  2
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
order  3
Rejecting due to low SNR!
Rejecting due to low SNR!
order  4
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
order  5
Rejecting due to low SNR!
order  6
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
order  7
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
order  8
Rejecting due to low SNR!
order  9
order  10
order  11
Rejecting due to low SNR!
order  12
order  13
order  14
order  15
order  16
order  17
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
Rejecting due to low SNR!
order  18
order  19
order  20
Rejecting due to low SNR!
order  21
Rejecting due to low SNR!
Rejecting 

   .
   PRIMITIVE: storeProcessedArc
   ----------------------------
      PRIMITIVE: storeCalibration
      ---------------------------
      Calibration stored as calibrations/processed_arc/arcs_sr_1x1_brs300_20220629_1x1_blue001_arc.fits
      .
   .
	Wrote arcs_sr_1x1_brs300_20220629_1x1_blue001_arc.fits in output directory

reduce completed successfully.


**Note the residual in the output from above**
- Ideally residual <= 0.005, but <= 0.007 is also acceptable



Check the file: `wave_resid_std_blue.txt`  
So once you have found a line that is bad, what you can do is go to your   
`GHOSTDR > ghostdr > ghost > lookups > Polyfit > ghost_thar_linelist_20220718.txt`  
find the bad line(s) and comment them out.  

Now when you rerun the cell above you should get a better residual.  

so close. 

# Generate Science Frames -> Extract your Object 

In [90]:
# ---------------------------- There is (theoretically) nothing to change in this cell

biasCorrect = 'calibrations/processed_bias/'+processedBias_nxmRed
extractProfile_slit = 'calibrations/processed_slit/'+processedSlit
extractProfile_flat = 'calibrations/processed_flat/'+processedFlat_Red
extractProfile_slitflat = 'calibrations/processed_slitflat/'+processedSlitFlat
addWavelengthSol = 'calibrations/processed_arc/'+processedArc_Red

primitives = 'biasCorrect:bias={} extractProfile:slit={} extractProfile:flat={} extractProfile:slitflat={} addWavelengthSolution={} barycentricCorrect:correction_factor=1.0 responseCorrect:skip=True'.format(biasCorrect, extractProfile_slit, extractProfile_flat, extractProfile_slitflat, addWavelengthSol) 


# ------------------------------------------ 
# Loop through all of the red frames for your science object and reduce them all
for file in glob.glob('raw/'+ scienceObj +'*red*.fits'):

    l2 = 'reduce --drpkg ghostdr -p {} {}'.format(primitives, file)
           
    # -------------- Add above commands to a script and run it
    file  = open('./script.sh','w')
    file.write('#!/bin/sh \nsource activate dragons\n'+l2)
    file.close()
    os.chmod('script.sh', stat.S_IRWXU )

    _ = subprocess.call('./script.sh')


# ------------------------------------------------------------------------------------
# Now do it again for blue
biasCorrect = 'calibrations/processed_bias/'+processedBias_nxmBlue
extractProfile_slit = 'calibrations/processed_slit/'+processedSlit
extractProfile_flat = 'calibrations/processed_flat/'+processedFlat_Blue
extractProfile_slitflat = 'calibrations/processed_slitflat/'+processedSlitFlat
addWavelengthSol = 'calibrations/processed_arc/'+processedArc_Blue

primitives = 'biasCorrect:bias={} extractProfile:slit={} extractProfile:flat={} extractProfile:slitflat={} addWavelengthSolution={} barycentricCorrect:correction_factor=1.0 responseCorrect:skip=True'.format(biasCorrect, extractProfile_slit, extractProfile_flat, extractProfile_slitflat, addWavelengthSol) 


# ------------------------------------------ 
# Loop through all of the red frames for your science object and reduce them all
for file in glob.glob('raw/'+ scienceObj +'*blue*.fits'):

    l2 = 'reduce --drpkg ghostdr -p {} {}'.format(primitives, file)
           
    # -------------- Add above commands to a script and run it
    file  = open('./script.sh','w')
    file.write('#!/bin/sh \nsource activate dragons\n'+l2)
    file.close()
    os.chmod('script.sh', stat.S_IRWXU )

    _ = subprocess.call('./script.sh')


# ------------------------------------------------------------------------------------
# This produced a processed red and blue frames for your object

# -------------- Create a folder called output if its not already been created
if os.path.isdir('output') == False:
    os.mkdir('output')

# -------------- Note the name
processedObj =  glob.glob(scienceObj+'*.fits')

# -------------- Move the processed files into output
for obj in processedObj:
    if os.path.isfile(os.path.join('output/', obj)):
        os.remove(os.path.join('output/', obj))
    shutil.move(obj, 'output/')


			--- reduce v3.0.3 ---

Running on Python 3.7.13
All submitted files appear valid:
raw/aqu2_sr_2x4_br1800s300_20220629_2x4_red003.fits
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/opt/anaconda3/envs/dragons/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


OVERRIDING GHOST CALIBRATIONS


RECIPE: reduce
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   .
   PRIMITIVE: addVAR
   -----------------
   Adding the read noise component of the variance
   .
   PRIMITIVE: overscanCorrect
   --------------------------
      PRIMITIVE: subtractOverscan
      ---------------------------
      .
      PRIMITIVE: trimOverscan
     

Creating order: 0
Creating order: 1
Creating order: 2
Creating order: 3
Creating order: 4
Creating order: 5
Creating order: 6
Creating order: 7
Creating order: 8
Creating order: 9
Creating order: 10
Creating order: 11
Creating order: 12
Creating order: 13
Creating order: 14
Creating order: 15
Creating order: 16
Creating order: 17
Creating order: 18
Creating order: 19
Creating order: 20
Creating order: 21
Creating order: 22
Creating order: 23
Creating order: 24
Creating order: 25
Creating order: 26
Creating order: 27
Creating order: 28
Creating order: 29
Creating order: 30
Creating order: 31
Creating order: 32
OBJECTS:[0, 1]
SKY:False
Extracting order: 0
Extracting order: 1
Extracting order: 2
Extracting order: 3
Extracting order: 4
Extracting order: 5
Extracting order: 6
Extracting order: 7
Extracting order: 8
Extracting order: 9
Extracting order: 10
Extracting order: 11
Extracting order: 12
Extracting order: 13
Extracting order: 14
Extracting order: 15
Extracting order: 16
Extracting 

   .
   PRIMITIVE: flatCorrect
   ----------------------
   Skipping the flat field correction step
   .
   PRIMITIVE: addWavelengthSolution
   --------------------------------
      PRIMITIVE: getProcessedArc
      --------------------------
         PRIMITIVE: getCalibration
         -------------------------
         Received calibration request for aqu2_sr_2x4_br1800s300_20220629_2x4_red003_extractedProfile.fits
         LOCAL CALIBRATION SEARCH:
         Cached calibration calibrations/processed_arc/arcs_sr_1x1_brs300_20220629_1x1_red001_arc.fits matched.
         .
      Using calibrations/processed_arc/arcs_sr_1x1_brs300_20220629_1x1_red001_arc.fits for aqu2_sr_2x4_br1800s300_20220629_2x4_red003_extractedProfile.fits
      .
      PRIMITIVE: getProcessedArc
      --------------------------
         PRIMITIVE: getCalibration
         -------------------------
         Received calibration request for aqu2_sr_2x4_br1800s300_20220629_2x4_red003_extractedProfile.fits
         LOCAL 

...

   .
   PRIMITIVE: flatCorrect
   ----------------------
   Skipping the flat field correction step
   .
   PRIMITIVE: addWavelengthSolution
   --------------------------------
      PRIMITIVE: getProcessedArc
      --------------------------
         PRIMITIVE: getCalibration
         -------------------------
         Received calibration request for aqu2_sr_2x4_br1800s300_20220629_2x4_blue002_extractedProfile.fits
         LOCAL CALIBRATION SEARCH:
         Cached calibration calibrations/processed_arc/arcs_sr_1x1_brs300_20220629_1x1_blue001_arc.fits matched.
         .
      Using calibrations/processed_arc/arcs_sr_1x1_brs300_20220629_1x1_blue001_arc.fits for aqu2_sr_2x4_br1800s300_20220629_2x4_blue002_extractedProfile.fits
      .
      PRIMITIVE: getProcessedArc
      --------------------------
         PRIMITIVE: getCalibration
         -------------------------
         Received calibration request for aqu2_sr_2x4_br1800s300_20220629_2x4_blue002_extractedProfile.fits
         L

OVERRIDING GHOST CALIBRATIONS


RECIPE: reduce
   PRIMITIVE: prepare
   ------------------
      PRIMITIVE: validateData
      -----------------------
      .
      PRIMITIVE: standardizeStructure
      -------------------------------
      .
      PRIMITIVE: standardizeHeaders
      -----------------------------
         PRIMITIVE: standardizeObservatoryHeaders
         ----------------------------------------
         Updating keywords that are common to all Gemini data
         .
         PRIMITIVE: standardizeInstrumentHeaders
         ---------------------------------------
         .
      .
      PRIMITIVE: standardizeWCS
      -------------------------
      .
   .
   PRIMITIVE: addDQ
   ----------------
   WARNING - No static BPMs defined
   .
   PRIMITIVE: addVAR
   -----------------
   Adding the read noise component of the variance
   .
   PRIMITIVE: overscanCorrect
   --------------------------
      PRIMITIVE: subtractOverscan
      ---------------------------
      .
      PRIMITIVE: trimOverscan
     

Creating order: 0
Creating order: 1
Creating order: 2
Creating order: 3
Creating order: 4
Creating order: 5
Creating order: 6
Creating order: 7
Creating order: 8
Creating order: 9
Creating order: 10
Creating order: 11
Creating order: 12
Creating order: 13
Creating order: 14
Creating order: 15
Creating order: 16
Creating order: 17
Creating order: 18
Creating order: 19
Creating order: 20
Creating order: 21
Creating order: 22
Creating order: 23
Creating order: 24
Creating order: 25
Creating order: 26
Creating order: 27
Creating order: 28
Creating order: 29
Creating order: 30
Creating order: 31
Creating order: 32
Creating order: 33
Creating order: 34
OBJECTS:[0, 1]
SKY:False
Extracting order: 0
Extracting order: 1
Extracting order: 2
Extracting order: 3
Extracting order: 4
Extracting order: 5
Extracting order: 6
Extracting order: 7
Extracting order: 8
Extracting order: 9
Extracting order: 10
Extracting order: 11
Extracting order: 12
Extracting order: 13
Extracting order: 14
Extracting orde

   .
   PRIMITIVE: flatCorrect
   ----------------------
   Skipping the flat field correction step
   .
   PRIMITIVE: addWavelengthSolution
   --------------------------------
      PRIMITIVE: getProcessedArc
      --------------------------
         PRIMITIVE: getCalibration
         -------------------------
         Received calibration request for aqu2_sr_2x4_br1800s300_20220629_2x4_blue003_extractedProfile.fits
         LOCAL CALIBRATION SEARCH:
         Cached calibration calibrations/processed_arc/arcs_sr_1x1_brs300_20220629_1x1_blue001_arc.fits matched.
         .
      Using calibrations/processed_arc/arcs_sr_1x1_brs300_20220629_1x1_blue001_arc.fits for aqu2_sr_2x4_br1800s300_20220629_2x4_blue003_extractedProfile.fits
      .
      PRIMITIVE: getProcessedArc
      --------------------------
         PRIMITIVE: getCalibration
         -------------------------
         Received calibration request for aqu2_sr_2x4_br1800s300_20220629_2x4_blue003_extractedProfile.fits
         L

Done!